In [1]:
#!pip install requests

In [2]:
#!pip install xlwings

In [3]:
# Referenced from this youtube video:
# https://www.youtube.com/watch?v=CZUy_MCrA-E
# https://www.youtube.com/watch?v=rguE-1Nl1qg


In [4]:
import requests
import pandas as pd
import time
import xlwings as xw
from bs4 import BeautifulSoup
import datetime

import tkinter as tk
from tkinter import ttk

In [5]:
TWO_PERCENT_MARKET_PRICE = 0.0

In [6]:
# get the last thursday of the current month
def last_thursday_version_2(year, month):
    
    if month == 1 or month == 2 or month == 3 or month == 4 or month == 5 or month == 6 or month == 7 or month == 8 or month == 9:
        date = f"{year}-0{month}-01"
    if month == 10 or month == 11 or month == 12:
        date = f"{year}-{month}-01"
        
    
    # we have a datetime series in our dataframe...
    df_Month = pd.to_datetime(date)
    
    # we can easily get the month's end date:
    df_mEnd = df_Month + pd.tseries.offsets.MonthEnd(1)
    
    # Thursday is weekday 3, so the offset for given weekday is
    offset = (df_mEnd.weekday() - 3) % 7

    # now to get the date of the last Thursday of the month, subtract it from 
    # month end date:
    df_Expiry = df_mEnd - pd.to_timedelta(offset, unit='D')

    return df_Expiry

In [7]:
# get the last thursday of the next month
def last_thursday(year, month):
    if month == 12:
        date = f"{year+1}-01-01"
    if month == 1 or month == 2 or month == 3 or month == 4 or month == 5 or month == 6 or month == 7 or month == 8:
        date = f"{year}-0{month+1}-01"
    if month == 9 or month == 10 or month == 11:
        date = f"{year}-{month+1}-01"
        
    
    # we have a datetime series in our dataframe...
    df_Month = pd.to_datetime(date)
    
    # we can easily get the month's end date:
    df_mEnd = df_Month + pd.tseries.offsets.MonthEnd(1)
    
    # Thursday is weekday 3, so the offset for given weekday is
    offset = (df_mEnd.weekday() - 3) % 7

    # now to get the date of the last Thursday of the month, subtract it from 
    # month end date:
    df_Expiry = df_mEnd - pd.to_timedelta(offset, unit='D')

    return df_Expiry

In [8]:
# # Function to display DataFrames
# def display_dataframes(df1, df2):
#     # Create a new Tkinter window
#     window = tk.Tk()
#     window.title("Display DataFrames")

#     # Create a Text widget to display DataFrame 1
#     text1 = tk.Text(window, height=30, width=100)
#     text1.insert(tk.END, df1.to_string(index=False))
#     text1.pack(side=tk.LEFT, padx=10, pady=10)

#     # Create a Text widget to display DataFrame 2
#     text2 = tk.Text(window, height=30, width=100)
#     text2.insert(tk.END, df2.to_string(index=False))
#     text2.pack(side=tk.RIGHT, padx=10, pady=10)

#     window.mainloop()

In [9]:
# Reference from https://learndataanalysis.org/create-google-sheets-service-instances-in-python/
import pickle
import os
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request


def Create_Service(client_secret_file, api_name, api_version, *scopes):
    print(client_secret_file, api_name, api_version, scopes, sep='-')
    CLIENT_SECRET_FILE = client_secret_file
    API_SERVICE_NAME = api_name
    API_VERSION = api_version
    SCOPES = [scope for scope in scopes[0]]
    print(SCOPES)
    
    cred = None
    
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            cred = pickle.load(token)
    
    if not cred or not cred.valid:
        if cred and cred.expired and cred.refresh_token:
            cred.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            cred = flow.run_local_server()
        
        with open('token.pickle', 'wb') as token:
            pickle.dump(cred, token)
            
    try:
        service = build(API_SERVICE_NAME, API_VERSION, credentials=cred)
        print(API_SERVICE_NAME, 'service created successfully')
        return service
    except Exception as e:
        print(e)
        return None

In [ ]:
ticker = "VEDL"
exchange = "NSE"

def current_market_price(ticker, exchange):
    
    url = f"https://www.google.com/finance/quote/{ticker}:{exchange}"

    for _ in range(1000000):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        class1 ="YMlKec fxKbKc"
        
        price = float(soup.find(class_=class1).text.strip()[1:].replace(",",""))
        yield price
        
        time.sleep(5)


# https://www.nseindia.com/api/option-chain-equities?symbol=VEDL
"""
headers = {"accept-encoding": "gzip, deflate, br, zstd",
            "accept-language": "en-US,en;q=0.9",
            "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"

        }
"""
# https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY
"""
headers = {"accept-encoding": "gzip, deflate, br",
            "accept-language": "en-US,en;q=0.9",
            "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"

        }
"""

while True:
    try:
        
        url = f"https://www.nseindia.com/api/option-chain-equities?symbol={ticker}"
        headers = {"accept-encoding": "gzip, deflate, br, zstd",
                   "accept-language": "en-US,en;q=0.9",
                   "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"

        }

        session = requests.Session()
        data = session.get(url, headers=headers).json()["records"]["data"]
        ocdata = []
        for i in data:
            for j, k in i.items():
                if j == "CE" or j == "PE":
                    info = k
                    info["instrumentType"] = j
                    ocdata.append(info)

        df = pd.DataFrame(ocdata)
        wb = xw.Book("optionchaintracker.xlsx")
        st = wb.sheets("vedl")
        st.range("A1").value = df
        #print(df)
        
        for price in current_market_price(ticker, exchange):
            two_percent_cmp = price + 0.02 * price
            TWO_PERCENT_MARKET_PRICE = two_percent_cmp
            break
        
        print(TWO_PERCENT_MARKET_PRICE)
        
        
        # access dataframe for atm price
        atm = int(round(TWO_PERCENT_MARKET_PRICE/2.5,0)*2.5)
        print(atm)
        
        
        output_ce = pd.DataFrame()
        
        atm_pe = atm
        output_pe = pd.DataFrame()
        
        for _ in range(5):
            
            # (for ce)
            ab = True
            while ab:


                fd = df[df['strikePrice'] == atm]

                if fd.empty:
                    #print("empty df", atm)
                    atm = atm+0.5
                else:
                    ab = False

            #print(fd)
            
            # (for pe)
            ab_pe = True
            while ab_pe:


                fd_pe = df[df['strikePrice'] == atm_pe]

                if fd_pe.empty:
                    #print("empty df", atm_pe)
                    atm_pe = atm_pe-0.5
                else:
                    ab_pe = False
            
            #print(fd_pe)



            # (for ce)convert expiry date in particular format
            fd = fd.reset_index()
            for i in range(len(fd)):
                expiry_date_str = fd["expiryDate"].iloc[i]
                temp_expiry = datetime.datetime.strptime(expiry_date_str, '%d-%b-%Y')
                result_expiry = temp_expiry.strftime('%d-%m-%Y')
                fd.at[i, "expiryDate"]= result_expiry
            #print(fd)
            #print(type(fd["expiryDate"].iloc[0]))
            
            
            # (for pe) convert expiry date in particular format
            fd_pe = fd_pe.reset_index()
            for i in range(len(fd_pe)):
                expiry_date_str_pe = fd_pe["expiryDate"].iloc[i]
                temp_expiry_pe = datetime.datetime.strptime(expiry_date_str_pe, '%d-%b-%Y')
                result_expiry_pe = temp_expiry_pe.strftime('%d-%m-%Y')
                fd_pe.at[i, "expiryDate"]= result_expiry_pe

                
                
            # (last thursday (CE))
            today_year = datetime.datetime.now().year
            today_month = datetime.datetime.now().month
            today_day = datetime.datetime.now().day
            l_thursday = last_thursday_version_2(today_year, today_month)
            
            # if today is within the last thursday, then we will take current month last thursday as an adjusted_expiry
            if today_day < l_thursday.day or today_day == l_thursday.day and today_month == l_thursday.month:
                adjusted_expiry = last_thursday_version_2(today_year, today_month)
                print("in first if")
            # if today is greater than last thursday, then we will take next month last thursday as an adjusted_expiry
            if today_day > l_thursday.day and today_month == l_thursday.month:
                adjusted_expiry = last_thursday(today_year, today_month)
                print("in second if")
            
            format = '%d-%m-%Y'
            adjusted_expiry = adjusted_expiry.strftime(format)
            #year, month, day = str(l_thursday.date()).split("-")
            #l_thursday = f"{day}-{month}-{year}"
            #print(l_thursday)
            #print(l_thursday.date())
            #print(str(l_thursday.date()))
            #print(type(l_thursday.date()))
            #print(type(str(l_thursday.date())))
            
            # (last thursday (PE))
            today_year_pe = datetime.datetime.now().year
            today_month_pe = datetime.datetime.now().month
            today_day_pe = datetime.datetime.now().day
            l_thursday_pe = last_thursday_version_2(today_year_pe, today_month_pe)
            
            # if today is within the last thursday, then we will take current month last thursday as an adjusted_expiry
            if today_day_pe < l_thursday_pe.day or today_day_pe == l_thursday_pe.day and today_month_pe == l_thursday_pe.month:
                adjusted_expiry_pe = last_thursday_version_2(today_year_pe, today_month_pe)
                print("in first if")
            # if today is greater than last thursday, then we will take next month last thursday as an adjusted_expiry
            if today_day_pe > l_thursday_pe.day and today_month_pe == l_thursday_pe.month:
                adjusted_expiry_pe = last_thursday(today_year_pe, today_month_pe)
                print("in second if")
            
            format = '%d-%m-%Y'
            adjusted_expiry_pe = adjusted_expiry_pe.strftime(format)
            #year_pe, month_pe, day_pe = str(l_thursday_pe.date()).split("-")
            #l_thursday_pe = f"{day_pe}-{month_pe}-{year_pe}"
            
            
            # (subset_ce (CE))
            subset_ce = fd[(fd.instrumentType == "CE") & (fd.expiryDate == adjusted_expiry)]
            #print(subset_ce)
            output_ce = pd.concat([output_ce, subset_ce])
            
            
            # (subset_pe (PE))
            subset_pe = fd_pe[(fd_pe.instrumentType == "PE") & (fd_pe.expiryDate == adjusted_expiry_pe)]
            #print(subset_pe)
            output_pe = pd.concat([output_pe, subset_pe])
            
            #(for CE)
            atm += 2.5
            
            #(for PE)
            atm_pe -= 2.5
        
        output_ce = output_ce[["strikePrice", "expiryDate", "lastPrice", "instrumentType"]]
        output_pe = output_pe[["strikePrice", "expiryDate", "lastPrice", "instrumentType"]]
        print(output_ce)
        print(output_pe)
        
        # Reference from https://learndataanalysis.org/export-pandas-dataframe-to-google-sheets-in-python/
        CLIENT_SECRET_FILE = 'client_secret.json'
        API_SERVICE_NAME = 'sheets'
        API_VERSION = 'v4'
        SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
        gsheetId = "1mZSb2ssmSnHX-5cJFWsYUR4xGlr_oaKb-22OIIQ36oY"

        service = Create_Service(CLIENT_SECRET_FILE, API_SERVICE_NAME, API_VERSION, SCOPES)

        #d = {'col1': [1, 2], 'col2': [3, 4]}
        #df = pd.DataFrame(data=d)
        #print(df)
        print(service)
        response_date = service.spreadsheets().values().append(
            spreadsheetId = gsheetId,
            valueInputOption = 'RAW',
            range= 'DF!D2',
            body = dict(
                majorDimension = 'ROWS',
                values = output_ce.T.reset_index().T.values.tolist())

            ).execute()
        
        response_date = service.spreadsheets().values().append(
            spreadsheetId = gsheetId,
            valueInputOption = 'RAW',
            range= 'DF!H2',
            body = dict(
                majorDimension = 'ROWS',
                values = output_pe.T.reset_index().T.values.tolist())

            ).execute()
        #display_dataframes(output_ce, output_pe)
        #print("after display dataframes")
        
        time.sleep(300)
    except Exception as e:
        pass

278.613
277
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in first if
   strikePrice  expiryDate  lastPrice instrumentType
0        277.5  28-03-2024       1.00             CE
0        280.0  28-03-2024       0.55             CE
0        282.5  28-03-2024       0.30             CE
1        285.0  28-03-2024       0.20             CE
0        287.5  28-03-2024       0.10             CE
   strikePrice  expiryDate  lastPrice instrumentType
1        275.0  28-03-2024       3.10             PE
0        272.5  28-03-2024       1.65             PE
1        270.0  28-03-2024       0.60             PE
1        267.5  28-03-2024       0.40             PE
0        265.0  28-03-2024       0.25             PE
client_secret.json-sheets-v4-(['https://www.googleapis.com/auth/spreadsheets'],)
['https://www.googleapis.com/auth/spreadsheets']
278.613
277
in first if
in first if
in first if
in first if
in first if
in first if
in first if
in fir